<a href="https://colab.research.google.com/github/GarvJain2205/Factorial24_FairStack/blob/main/Hotel_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# 1. Install necessary libraries
!pip install streamlit pyngrok -q
import pandas as pd
import os

# 2. IMPORTANT: Upload your 'hotel_bookings.csv' file using the Colab file explorer before running this cell.
print("✅ Libraries installed. Make sure 'hotel_bookings.csv' is uploaded.")

# 3. Define the Streamlit app code as a Python string
app_code = """
import streamlit as st
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from statsmodels.tsa.arima.model import ARIMA
import warnings

# --- Page Configuration and Style ---
st.set_page_config(
    page_title="Hotel Booking Analysis Dashboard",
    page_icon="🏨",
    layout="wide"
)
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

# --- Caching Functions for Performance ---
@st.cache_data
def load_data(path):
    df = pd.read_csv(path)
    # Data Cleaning
    month_map = {
        'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
        'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
    }
    df['arrival_date_month_numeric'] = df['arrival_date_month'].map(month_map)
    date_cols = ['arrival_date_year', 'arrival_date_month_numeric', 'arrival_date_day_of_month']
    for col in date_cols:
        df[col] = df[col].fillna(0)
    df['arrival_date'] = pd.to_datetime(
        df['arrival_date_year'].astype(int).astype(str) + '-' +
        df['arrival_date_month_numeric'].astype(int).astype(str) + '-' +
        df['arrival_date_day_of_month'].astype(int).astype(str),
        errors='coerce'
    )
    df.dropna(subset=['arrival_date'], inplace=True)
    df_valid = df[df['is_canceled'] == 0].copy()
    return df, df_valid

@st.cache_resource
def train_cancellation_model(_df):
    features = [
        'lead_time', 'arrival_date_week_number', 'adults', 'children', 'babies',
        'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled',
        'booking_changes', 'adr', 'required_car_parking_spaces', 'total_of_special_requests',
        'hotel', 'deposit_type', 'customer_type'
    ]
    target = 'is_canceled'
    df_model = _df.copy()
    df_model.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_model.dropna(subset=features + [target], inplace=True)
    df_model = pd.get_dummies(df_model, columns=['hotel', 'deposit_type', 'customer_type'], drop_first=True)
    encoded_features = [col for col in df_model.columns if any(c in col for c in features)]
    X = df_model[encoded_features]
    y = df_model[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model_rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_depth=10)
    model_rf.fit(X_train, y_train)
    return model_rf, X_test, y_test, _df

# --- Main App ---
st.title("🏨 Hotel Booking Analysis & Forecasting Dashboard")
df, df_valid = load_data('hotel_bookings.csv')

st.sidebar.header("Dashboard Controls")
page = st.sidebar.radio("Select an Analysis:",
                        ("Occupancy Forecast", "Revenue Forecast", "Cancellation Analysis"))

if page == "Occupancy Forecast":
    st.header("Future Occupancy Forecast (Prophet)")
    forecast_days = st.slider("Select forecast days:", 30, 365, 90, key="occupancy_slider")
    occupancy_df = df_valid.groupby('arrival_date').size().reset_index(name='y')
    occupancy_df.rename(columns={'arrival_date': 'ds'}, inplace=True)
    model_prophet = Prophet(daily_seasonality=False, yearly_seasonality=True, weekly_seasonality=True)
    model_prophet.fit(occupancy_df)
    future_dates = model_prophet.make_future_dataframe(periods=forecast_days)
    forecast_prophet = model_prophet.predict(future_dates)

    st.subheader("Occupancy Forecast Plot")
    fig1 = model_prophet.plot(forecast_prophet)
    st.pyplot(fig1)

    st.subheader("Key Insights")
    forecast_period = forecast_prophet.tail(forecast_days)
    total_bookings = forecast_period['yhat'].sum()
    avg_bookings = forecast_period['yhat'].mean()
    peak_day_row = forecast_period.loc[forecast_period['yhat'].idxmax()]
    peak_date = peak_day_row['ds'].strftime('%B %d, %Y')
    peak_bookings = peak_day_row['yhat']

    st.markdown(
f'''
**Chart Legend:**
- **Black Dots**: Represent the actual, historical daily bookings.
- **Blue Line**: The model's prediction for future bookings.
- **Light Blue Area**: The confidence interval. We are 95% confident that the actual number of bookings will fall within this range.

**Numerical Summary for the Next {forecast_days} Days:**
- **Total Forecasted Bookings:** We project a total of **{total_bookings:,.0f}** new bookings.
- **Average Daily Occupancy:** We expect an average of **{avg_bookings:.0f}** bookings per day.
- **Peak Demand:** The busiest day is forecasted to be **{peak_date}** with **{peak_bookings:.0f}** bookings.
'''
    )

elif page == "Revenue Forecast":
    st.header("Future Revenue Forecast (ARIMA)")
    forecast_days_rev = st.slider("Select forecast days:", 30, 365, 90, key="revenue_slider")
    df_valid['total_nights'] = df_valid['stays_in_weekend_nights'] + df_valid['stays_in_week_nights']
    df_valid['revenue'] = df_valid['adr'] * df_valid['total_nights']
    revenue_ts = df_valid.groupby('arrival_date')['revenue'].sum().asfreq('D', fill_value=0)
    with st.spinner('Training ARIMA model...'):
        model_arima = ARIMA(revenue_ts, order=(5, 1, 0))
        arima_fit = model_arima.fit()
    forecast_arima = arima_fit.get_forecast(steps=forecast_days_rev)
    forecast_index = pd.date_range(start=revenue_ts.index[-1] + pd.Timedelta(days=1), periods=forecast_days_rev)
    predicted_revenue = forecast_arima.predicted_mean
    confidence_intervals = forecast_arima.conf_int()

    st.subheader("Revenue Forecast Plot")
    fig2, ax = plt.subplots(figsize=(15, 7))
    ax.plot(revenue_ts.tail(365), label='Historical Daily Revenue')
    ax.plot(forecast_index, predicted_revenue, color='darkgreen', label='Predicted Revenue')
    ax.fill_between(forecast_index, confidence_intervals.iloc[:, 0], confidence_intervals.iloc[:, 1], color='k', alpha=.15)
    ax.legend()
    st.pyplot(fig2)

    st.subheader("Financial Projections")
    total_revenue = predicted_revenue.sum()
    avg_revenue = predicted_revenue.mean()
    lower_bound_total = confidence_intervals.iloc[:, 0].sum()
    upper_bound_total = confidence_intervals.iloc[:, 1].sum()

    st.markdown(
f'''
**Chart Legend:**
- **Blue Line**: The actual, historical daily revenue.
- **Green Line**: The model's forecast for future revenue.
- **Grey Area**: The 95% confidence interval for the revenue forecast.

**Numerical Summary for the Next {forecast_days_rev} Days:**
- **Projected Total Revenue:** We forecast **${total_revenue:,.2f}**.
- **Average Daily Revenue:** We expect an average of **${avg_revenue:,.2f}** per day.
- **Confidence Range:** The total revenue will likely fall between **${lower_bound_total:,.2f} and ${upper_bound_total:,.2f}**.
'''
    )

elif page == "Cancellation Analysis":
    st.header("Cancellation Likelihood Analysis")
    with st.spinner('Training classification model...'):
        model_rf, X_test, y_test, df_full = train_cancellation_model(df)

    st.subheader("Distribution of Cancellation Likelihood")
    cancellation_prob = model_rf.predict_proba(X_test)[:, 1]
    cancellation_output = df_full.loc[X_test.index].copy()
    cancellation_output['Cancellation_Probability'] = cancellation_prob
    fig3, ax = plt.subplots(figsize=(12, 6))
    sns.histplot(data=cancellation_output, x='Cancellation_Probability', kde=True, bins=30, color='coral', ax=ax)
    ax.axvline(x=0.5, color='red', linestyle='--', label='50% Probability Threshold')
    ax.legend()
    st.pyplot(fig3)

    st.subheader("Cancellation Risk Assessment")
    total_analyzed = len(X_test)
    high_risk_threshold = st.slider("Set high-risk threshold:", 0.50, 1.00, 0.75, 0.05)
    high_risk_bookings = cancellation_output[cancellation_output['Cancellation_Probability'] > high_risk_threshold]
    num_high_risk = len(high_risk_bookings)
    pct_high_risk = (num_high_risk / total_analyzed) * 100 if total_analyzed > 0 else 0

    st.markdown(
f'''
**Chart Legend:**
- **X-axis**: The predicted probability of a booking being canceled (0% to 100%).
- **Y-axis**: The number of bookings in each risk category.
- **Red Line**: A 50% probability threshold for reference.

**Numerical Summary for the {total_analyzed:,} Upcoming Bookings:**
- **High-Risk Bookings:** Using a {high_risk_threshold:.0%} risk threshold, we have identified **{num_high_risk}** bookings (**{pct_high_risk:.1f}%** of the total) that are highly likely to cancel.
- **Recommendation:** Target these **{num_high_risk}** bookings with a proactive engagement campaign to mitigate revenue loss.
'''
    )
    st.dataframe(high_risk_bookings[['arrival_date', 'lead_time', 'adr', 'customer_type', 'Cancellation_Probability']])

    st.subheader("Model Performance")
    y_pred = model_rf.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    st.dataframe(pd.DataFrame(report).transpose())

"""

# 4. Write the app code to a file
with open('app.py', 'w') as f:
    f.write(app_code)

# 5. Set up the ngrok tunnel and run the Streamlit app
from pyngrok import ngrok

# Terminate open tunnels if they exist
ngrok.kill()

NGROK_AUTH_TOKEN = "33jcg6JVsaygEzWZDtoqNjgbbF8_rDeRWR7wv47YUGVPhM7p"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open a http tunnel on the default streamlit port (8501)
public_url = ngrok.connect(8501)
print(f"🎈 Your Streamlit dashboard is live at: {public_url}")

# Run the app in the background
!streamlit run app.py &>/dev/null&

✅ Libraries installed. Make sure 'hotel_bookings.csv' is uploaded.
🎈 Your Streamlit dashboard is live at: NgrokTunnel: "https://unstolen-sherron-hierarchically.ngrok-free.dev" -> "http://localhost:8501"
